In [47]:
import os

Project = os.path.abspath(os.path.join(os.getcwd(), 'Data')) + '/' # Get the parent directory
print(Project)

D:\Google Drive - Columbia University\AI4Finance\Imitation Learning\FinRL_Imitation_Learning\Data/


In [48]:
import numpy as np
import pandas as pd
from scipy import stats
import scipy.optimize as sco
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, PooledOLS, FamaMacBeth
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
import seaborn as sns; sns.set()
from pandas.tseries.offsets import *
from dateutil.relativedelta import *
import datetime as dt
import os
from IPython.core.pylabtools import figsize
from IPython.core.interactiveshell import InteractiveShell

import warnings
warnings.filterwarnings('ignore')

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [49]:
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:,.3f}'.format
pd.set_option('use_inf_as_na', True)
pd.set_option('display.max_columns', 100)
idx = pd.IndexSlice
figsize(18,6)

In [50]:
# 43 daily trading indicators
indicators = pd.read_sas(Project+'trading_indicators.sas7bdat')
indicators.columns = indicators.columns.str.lower()
indicators['permno'] = indicators['permno'].astype(int)
indicators.tail()


,permno,date,autocorr,buyminusselltrades,buyminusselltrades_abs,buyminussellvol,buyminussellvol_abs,effectivespread_percent_dw,hindex,overnightproduct,percentpriceimpact_lr_dw,percentrealizedspread_lr_dw,priceimpactintraday,quotedspread_percent_tw,realizedvol,realizedvolintraday,askhi,baspread,bidlo,hlspread,ivol_q,ivol_t,me,n30_pos,n5_pos,prc,ret,ret_abs,ret_am,ret_cto,ret_cto_abs,ret_otc,ret_otc_abs,ret_pm,total_n_trades_a,total_n_trades_m,turnover,turnover_am,var_ratio1,var_ratio2,turnover_pm,buyminussellvwap,priceimpactovernight,tsignsqrtdvol1,tsignsqrtdvol2
14663301,93436,2021-12-27,1.225,2.070,1.670,0.030,-0.429,-0.569,-0.489,0.185,-0.517,-0.344,-0.178,-0.604,-0.391,-0.491,8.105,-0.475,8.318,-0.328,-0.244,-0.250,8.961,0.275,0.501,8.242,0.516,-0.166,0.689,0.682,0.066,0.082,-0.762,-0.660,6.323,4.113,0.320,0.505,-2.146,-1.324,0.085,-0.772,-0.119,-0.286,-0.311
14663302,93436,2021-12-28,-1.065,2.289,3.012,0.263,-0.336,-0.570,-0.451,0.088,-0.529,-0.336,-0.178,-0.602,-0.531,-0.503,8.156,-0.445,8.109,-0.430,-0.244,-0.252,8.948,0.275,0.501,7.982,-0.130,-0.717,-0.219,0.245,-0.476,-0.336,-0.428,-0.085,6.614,4.140,0.193,0.359,-1.382,-1.525,-0.010,3.539,-0.119,-0.284,-0.310
14663303,93436,2021-12-29,0.719,2.652,3.339,-0.058,-0.348,-0.577,-0.594,0.084,-0.526,-0.347,-0.178,-0.601,-0.705,-0.668,7.962,-0.458,8.034,-0.441,-0.244,-0.252,8.969,0.275,0.501,7.981,-0.068,-0.796,-0.491,-0.018,-0.802,-0.091,-0.740,-0.103,6.441,3.941,0.147,0.248,-2.139,-0.425,0.028,4.094,-0.119,-0.288,-0.313
14663304,93436,2021-12-30,0.459,2.374,3.003,0.155,-0.435,-0.578,-0.464,0.248,-0.532,-0.344,-0.178,-0.603,-0.490,-0.570,7.992,-0.474,8.073,-0.382,-0.244,-0.252,8.925,0.275,0.501,8.000,-0.335,-0.456,0.700,-0.649,-0.073,0.121,-0.713,-0.676,4.328,3.897,0.039,0.155,-1.969,-0.738,-0.083,6.079,-0.119,-0.288,-0.313
14663305,93436,2021-12-31,-0.255,-1.380,0.833,0.127,-0.461,-0.576,-0.579,0.076,-0.536,-0.338,-0.178,-0.606,-0.855,-0.845,7.933,-0.455,8.058,-0.659,-0.244,-0.252,8.955,0.275,0.501,7.957,-0.294,-0.509,-0.105,0.059,-0.706,-0.404,-0.341,-0.400,3.303,3.220,-0.046,-0.007,-1.224,-0.758,-0.098,4.606,-0.119,-0.292,-0.317


In [51]:
features = indicators.columns.drop(['permno','date'])
features

Index(['autocorr', 'buyminusselltrades', 'buyminusselltrades_abs',
       'buyminussellvol', 'buyminussellvol_abs', 'effectivespread_percent_dw',
       'hindex', 'overnightproduct', 'percentpriceimpact_lr_dw',
       'percentrealizedspread_lr_dw', 'priceimpactintraday',
       'quotedspread_percent_tw', 'realizedvol', 'realizedvolintraday',
       'askhi', 'baspread', 'bidlo', 'hlspread', 'ivol_q', 'ivol_t', 'me',
       'n30_pos', 'n5_pos', 'prc', 'ret', 'ret_abs', 'ret_am', 'ret_cto',
       'ret_cto_abs', 'ret_otc', 'ret_otc_abs', 'ret_pm', 'total_n_trades_a',
       'total_n_trades_m', 'turnover', 'turnover_am', 'var_ratio1',
       'var_ratio2', 'turnover_pm', 'buyminussellvwap', 'priceimpactovernight',
       'tsignsqrtdvol1', 'tsignsqrtdvol2'],
      dtype='object')

In [52]:
# retail market order imbalance, moribvol_pctl is the 100 groups by sorting on moribvol at each date
retail = pd.read_sas(Project+'retail_market_order.sas7bdat', encoding = 'UTF-8')
retail.columns = retail.columns.str.lower()
retail[['permno','moribvol_pctl']] = retail[['permno','moribvol_pctl']].astype(int)
retail = retail[retail['date'] >= '2010-01-01'].reset_index(drop = True)
retail.tail()

,permno,ticker,companyname,date,russellgroup,sector,price,marketcap,moribvol,moribvol_pctl
12066442,93436,TSLA,TESLA INC,2022-12-23,1.000,XLY,123.150,"388,877.164",-0.111,38
12066443,93436,TSLA,TESLA INC,2022-12-27,1.000,XLY,109.100,"344,510.738",0.022,65
12066444,93436,TSLA,TESLA INC,2022-12-28,1.000,XLY,112.710,"355,910.225",-0.092,48
12066445,93436,TSLA,TESLA INC,2022-12-29,1.000,XLY,121.820,"384,677.348",-0.043,57
12066446,93436,TSLA,TESLA INC,2022-12-30,1.000,XLY,123.180,"388,971.892",-0.025,53


In [53]:
# select the top (bottom) 5 percentile as strong buy (sell) by the retail investors
retail['Buy'] = np.where(retail['moribvol_pctl'] >= 95, 1, 0)
retail['Sell'] = np.where(retail['moribvol_pctl'] <= 5, 1, 0)

In [54]:
# Forming training-testing dataset: pick up buy (sell) stock-day, and one day surrounding it without Buy
buy = retail[retail['Buy'] == 1][['permno','date','Buy']]
buy['event_date'] = buy['date']

nobuy1 = retail[retail['Buy'] == 0][['permno','date','Buy']]
nobuy1['event_date'] = nobuy1['date']+pd.offsets.BusinessDay(n=1)
nobuy1 = pd.merge(nobuy1, buy[['permno','event_date']], on = ['permno','event_date'], how = 'inner')

nobuy2 = retail[retail['Buy'] == 0][['permno','date','Buy']]
nobuy2['event_date'] = nobuy2['date']+pd.offsets.BusinessDay(n=-1)
nobuy2 = pd.merge(nobuy2, buy[['permno','event_date']], on = ['permno','event_date'], how = 'inner')

buy = pd.concat([buy, nobuy1, nobuy2], ignore_index = True).set_index(['permno','event_date','date'])\
      .sort_index().reset_index()
buy

,permno,event_date,date,Buy
0,10001,2010-02-23,2010-02-22,0
1,10001,2010-02-23,2010-02-23,1
2,10001,2010-02-23,2010-02-24,0
3,10001,2010-02-25,2010-02-24,0
4,10001,2010-02-25,2010-02-25,1
...,...,...,...,...
1853659,93435,2010-11-08,2010-11-09,0
1853660,93435,2011-03-29,2011-03-28,0
1853661,93435,2011-03-29,2011-03-29,1
1853662,93435,2011-03-30,2011-03-30,1


In [55]:
sample = pd.merge(buy.rename(columns = {'Buy':'ind'}), indicators.dropna(subset = features), 
                  on = ['permno','date'], how = 'inner')
sample.tail()

,permno,event_date,date,ind,autocorr,buyminusselltrades,buyminusselltrades_abs,buyminussellvol,buyminussellvol_abs,effectivespread_percent_dw,hindex,overnightproduct,percentpriceimpact_lr_dw,percentrealizedspread_lr_dw,priceimpactintraday,quotedspread_percent_tw,realizedvol,realizedvolintraday,askhi,baspread,bidlo,hlspread,ivol_q,ivol_t,me,n30_pos,n5_pos,prc,ret,ret_abs,ret_am,ret_cto,ret_cto_abs,ret_otc,ret_otc_abs,ret_pm,total_n_trades_a,total_n_trades_m,turnover,turnover_am,var_ratio1,var_ratio2,turnover_pm,buyminussellvwap,priceimpactovernight,tsignsqrtdvol1,tsignsqrtdvol2
831648,93435,2010-07-16,2010-07-19,0,-1.002,0.034,-0.455,0.121,-0.587,0.382,0.162,0.088,-0.142,0.469,-0.131,0.412,0.175,0.300,-0.556,0.173,-0.557,0.005,-0.024,-0.128,-0.318,-0.847,-1.356,-0.545,1.173,0.683,0.886,0.014,-0.682,1.590,1.158,1.987,-0.345,-0.428,-0.662,-0.659,1.073,1.049,-0.654,0.934,-0.114,0.400,0.414
831649,93435,2010-07-29,2010-07-30,0,0.421,0.088,-0.401,0.485,-0.198,-0.208,-0.315,0.088,-0.134,-0.152,-0.179,-0.260,-0.509,-0.469,-0.587,-0.337,-0.577,-0.503,0.081,-0.198,-0.319,-0.011,-0.909,-0.580,-0.343,-0.472,-0.599,-0.180,-0.584,-0.574,-0.125,-0.048,-0.345,-0.423,-0.588,-0.498,1.009,1.147,-0.749,-0.116,-0.151,-0.463,-0.555
831650,93435,2010-10-08,2010-10-07,0,-0.330,0.026,-0.463,0.150,-0.555,-0.097,-0.277,0.088,0.122,-0.183,-0.153,-0.153,-0.137,-0.089,-0.522,-0.191,-0.520,-0.133,-0.176,-0.137,-0.317,0.268,-1.081,-0.527,0.261,-0.472,0.535,0.404,-0.227,-0.018,-0.803,-0.311,-0.345,-0.426,-0.600,-0.471,0.606,0.206,-0.788,-0.427,0.115,0.213,0.208
831651,93435,2010-11-08,2010-11-08,1,0.195,0.180,-0.309,1.876,1.284,-0.269,-0.434,0.088,0.008,-0.297,-0.188,-0.206,0.000,0.106,-0.396,-0.330,-0.394,-0.265,-0.174,-0.207,-0.310,0.547,-0.427,-0.382,0.975,0.431,1.142,-0.041,-0.746,1.173,0.649,0.254,-0.325,-0.406,-0.202,0.088,0.735,0.670,-0.623,0.397,-0.170,0.296,0.248
831652,93435,2011-03-30,2011-03-30,1,-0.850,-0.006,-0.475,-0.070,-0.508,0.036,0.296,0.346,-0.090,0.088,-0.242,-0.187,0.609,0.485,-0.663,-0.294,-0.664,0.383,-0.033,-0.162,-0.335,0.198,-0.695,-0.671,-0.267,-0.624,-1.574,1.641,1.191,-1.371,0.835,-0.008,-0.336,-0.434,-0.380,-0.237,0.891,0.879,-0.626,0.719,-0.214,-0.101,-0.031


# NN

In [228]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [229]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            # Apply scaling if necessary
            if scale_data: 
                X = StandardScaler(with_mean=True, with_std=True).fit_transform(X)
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)
        else:
            self.X = torch.tensor(X).float()
            self.y = torch.tensor(y).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

    
class NeuralNet(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(state_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.l3 = nn.Linear(hidden_dim, action_dim)

    def forward(self, state):
        a = F.relu(self.l1(state))
        a = F.relu(self.l2(a))
        return F.softmax(self.l3(a))
    
    
def train_epoch(model,device,dataloader,loss_fn,optimizer):
    train_loss,train_correct=0.0,0
    model.train()
    for inputs, labels in dataloader:

        inputs,labels = inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, labels.type(torch.LongTensor))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        scores, predictions = torch.max(output.data, 1)
        train_correct += (predictions == labels).sum().item()

    return train_loss,train_correct
  
def valid_epoch(model,device,dataloader,loss_fn):
    valid_loss, val_correct = 0.0, 0
    model.eval()
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs,labels = inputs.to(device),labels.to(device)
            output = model(inputs)
            loss = loss_fn(output, labels.type(torch.LongTensor))
            valid_loss+=loss.item()
            scores, predictions = torch.max(output.data,1)
            val_correct+=(predictions == labels).sum().item()

    return valid_loss,val_correct

In [230]:
from sklearn.utils import shuffle
seed = 99
event_dates = sorted(sample['event_date'].unique())

len(event_dates)

3022

In [231]:
test_periods = event_dates[2100:]
len(test_periods)

922

In [232]:
train_periods = event_dates[:2100]
len(train_periods)

2100

In [233]:
data_test = sample[sample['event_date'].isin(test_periods)]
X_test = data_test[features].astype(np.float32).values
y_test = data_test['ind'].astype(np.float32).values

data_train = sample[sample['event_date'].isin(train_periods)]
X_train = data_train[features].astype(np.float32).values
y_train = data_train['ind'].astype(np.float32).values

In [234]:
# standardize
train_dataset = Dataset(X_train, y_train, scale_data=True)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True)

scaler = StandardScaler(with_mean=True, with_std=True).fit(X_train)
trade_dataset = Dataset(scaler.transform(X_test), y_test, scale_data=False)

# for CV
dataset = Dataset(X_train, y_train)

In [239]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
criterion = nn.CrossEntropyLoss()
num_epochs=10
batch_size=512
k=5

splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

In [240]:
history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    
    model = NeuralNet(state_dim=43, action_dim=2, hidden_dim=256)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=3e-4)
    
    if fold != 0: continue
        
    print('Fold {}'.format(fold + 1))

    for epoch in range(num_epochs):
        train_loss, train_correct=train_epoch(model,device,train_loader,criterion,optimizer)
        test_loss, test_correct=valid_epoch(model,device,test_loader,criterion)

        train_loss = train_loss
        train_acc = train_correct / len(train_loader.sampler) * 100
        test_loss = test_loss
        test_acc = test_correct / len(test_loader.sampler) * 100

        print("Epoch:{}/{} Training Loss:{:.3f} Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                             num_epochs,
                                                                                                             train_loss,
                                                                                                             test_loss,
                                                                                                             train_acc,
                                                                                                             test_acc))
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)   

NeuralNet(
  (l1): Linear(in_features=43, out_features=256, bias=True)
  (l2): Linear(in_features=256, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=2, bias=True)
)

Fold 1
Epoch:1/10 Training Loss:500.421 Test Loss:125.419 AVG Training Acc 62.76 % AVG Test Acc 62.54 %
Epoch:2/10 Training Loss:498.013 Test Loss:125.255 AVG Training Acc 62.97 % AVG Test Acc 62.60 %
Epoch:3/10 Training Loss:497.020 Test Loss:125.252 AVG Training Acc 63.04 % AVG Test Acc 62.63 %
Epoch:4/10 Training Loss:496.419 Test Loss:125.076 AVG Training Acc 63.11 % AVG Test Acc 62.72 %
Epoch:5/10 Training Loss:495.919 Test Loss:125.071 AVG Training Acc 63.20 % AVG Test Acc 62.70 %
Epoch:6/10 Training Loss:495.389 Test Loss:124.908 AVG Training Acc 63.27 % AVG Test Acc 62.72 %
Epoch:7/10 Training Loss:494.966 Test Loss:124.979 AVG Training Acc 63.34 % AVG Test Acc 62.62 %
Epoch:8/10 Training Loss:494.473 Test Loss:124.881 AVG Training Acc 63.44 % AVG Test Acc 62.73 %
Epoch:9/10 Training Loss:494.024 Test Loss:124.988 AVG Training Acc 63.49 % AVG Test Acc 62.68 %
Epoch:10/10 Training Loss:493.447 Test Loss:125.025 AVG Training Acc 63.60 % AVG Test Acc 62.71 %


NeuralNet(
  (l1): Linear(in_features=43, out_features=256, bias=True)
  (l2): Linear(in_features=256, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=2, bias=True)
)

NeuralNet(
  (l1): Linear(in_features=43, out_features=256, bias=True)
  (l2): Linear(in_features=256, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=2, bias=True)
)

NeuralNet(
  (l1): Linear(in_features=43, out_features=256, bias=True)
  (l2): Linear(in_features=256, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=2, bias=True)
)

NeuralNet(
  (l1): Linear(in_features=43, out_features=256, bias=True)
  (l2): Linear(in_features=256, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=2, bias=True)
)

In [241]:
y_pred = net(trade_dataset.X).detach().numpy()
res = sm.OLS(y_test, sm.add_constant(y_pred[:,1])).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1257.
Date:                Mon, 22 May 2023   Prob (F-statistic):          7.76e-275
Time:                        23:18:05   Log-Likelihood:            -2.3825e+05
No. Observations:              342100   AIC:                         4.765e+05
Df Residuals:                  342098   BIC:                         4.765e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3554      0.001    305.219      0.0

In [129]:
# net = NeuralNet(state_dim=43, 
#                 action_dim=2, 
#                 hidden_dim=256)

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)
# num_epochs = 200

# # Train the model
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for i, (inputs, targets) in enumerate(train_dataloader):
#         optimizer.zero_gra d()
#         outputs = net(inputs)
#         loss = criterion(outputs, targets.type(torch.LongTensor))        
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
    
#     # print statistics
#     print(f'Epoch: {epoch + 1}, loss: {running_loss:.3f}')
#     running_loss = 0.0

Epoch: 1, loss: 741.979
Epoch: 2, loss: 738.892
Epoch: 3, loss: 737.875
Epoch: 4, loss: 737.061
Epoch: 5, loss: 736.389
Epoch: 6, loss: 735.766
Epoch: 7, loss: 735.180
Epoch: 8, loss: 734.588
Epoch: 9, loss: 734.041
Epoch: 10, loss: 733.483
Epoch: 11, loss: 732.938
Epoch: 12, loss: 732.371
Epoch: 13, loss: 731.799
Epoch: 14, loss: 731.213
Epoch: 15, loss: 730.620
Epoch: 16, loss: 730.036
Epoch: 17, loss: 729.436
Epoch: 18, loss: 728.845
Epoch: 19, loss: 728.215
Epoch: 20, loss: 727.599
Epoch: 21, loss: 726.969
Epoch: 22, loss: 726.323
Epoch: 23, loss: 725.683
Epoch: 24, loss: 725.038
Epoch: 25, loss: 724.394
Epoch: 26, loss: 723.737
Epoch: 27, loss: 723.083
Epoch: 28, loss: 722.414
Epoch: 29, loss: 721.760
Epoch: 30, loss: 721.057
Epoch: 31, loss: 720.359
Epoch: 32, loss: 719.675
Epoch: 33, loss: 718.992
Epoch: 34, loss: 718.289
Epoch: 35, loss: 717.605
Epoch: 36, loss: 716.888
Epoch: 37, loss: 716.247
Epoch: 38, loss: 715.542
Epoch: 39, loss: 714.805
Epoch: 40, loss: 714.132
Epoch: 41